<a href="https://colab.research.google.com/github/TienNguyenMSBA/hotelbooking/blob/main/Group_Project_1__Advance_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### README

This notebook contains the code to predict final demand of a hotel booking data using different models:

1. Advance Booking (Pick-up) Models (Additive and Multiplicative)
2. Advance Booking Feedforward Neural Network Models
3. Time Series LSTM Models


In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [1]:
# from google.colab import files
# uploaded = files.upload()

file_path = '/content/hotel_bookings.csv'
df = pd.read_csv(file_path)

##### Read the data (csv file)

In [ ]:
df = pd.read_csv("hotel_bookings.csv")
df

##### **1. Pick-up models**

##### Convert date columns to datetime for filtering

In [ ]:
## Define features (X) and target (y)
feature_cols = ["days_prior", "bookings_on_hand", "day_of_week", "month", "naive_forecast"]
target_col = "final_demand"

def convert_datetime(df, columns):
    for col in columns:
        df[col] = pd.to_datetime(df[col])
    return df

date_columns = ["stay_date", "booking_date"]
df = convert_datetime(df, date_columns)

## Create feature columns used by models and aggregations
df["day_of_week"] = df["stay_date"].dt.day_name()
df["month"] = df["stay_date"].dt.month
df['days_prior_range'] = pd.cut(df['days_prior'], bins=[0,7,14,21,30], labels=['1-7','8-14','15-21','22-30'], right=True)
df['remaining_demand'] = df['final_demand'] - df['bookings_on_hand']

## Create naive forecast based on previous year's final_demand for same hotel and stay_date
df['stay_date'] = pd.to_datetime(df['stay_date'])
df = df.sort_values(['hotel_code', 'stay_date'])
# Create a shifted date column for lookup
df['prev_year_date'] = df['stay_date'] - pd.DateOffset(years=1)
# Build a mapping from (hotel_code, date) → final_demand
key_to_demand = dict(zip(zip(df['hotel_code'], df['stay_date']), df['final_demand']))
# Lookup previous year's value efficiently
df['naive_forecast'] = [
    key_to_demand.get((h, prev_date)) for h, prev_date in zip(df['hotel_code'], df['prev_year_date'])
]
# Drop missing
df = df.dropna(subset=['naive_forecast'])



##### **Helper functions** for pick-up models
- Additive model (bookings_on_hand + average remaining demand)
- Multiplicative model (bookings_on_hand/average booking rate)
- Calculate error metrics (MAE, MAPE)
- Compute MASE using naive forecast (previous year). For example, the naive forecast for stay date Apr 1st, 2010 is the actual demand of Apr 2nd, 2009
- Add the forecast to the validation set, using days_prior 1-30
- Print out the forecasting errors grouped by hotel code, days_prior range, and month

In [ ]:
## Additive model (bookings_on_hand + average remaining demand)
def add_model(df):
    df = df.copy()
    df = df[df['days_prior'].between(1,30)]
    df['average_remaining_demand'] = df.groupby(['hotel_code', 'days_prior'], sort=False)[
        'remaining_demand'].transform(np.mean)
    df['forecast'] = df['bookings_on_hand'] + df['average_remaining_demand']
    return df

## Multiplicative model (bookings_on_hand/average booking rate)
def multi_model(df):
    df = df.copy()
    df = df[df['days_prior'].between(1,30)]
    df['booking_rate'] = df['bookings_on_hand'] / df['final_demand']
    df['average_booking_rate'] = df.groupby(['hotel_code', 'days_prior'], sort=False)['booking_rate'].transform(np.mean)
    df['forecast'] = df['bookings_on_hand'] / df['average_booking_rate']

    # Handle inf/nan
    df['forecast'] = df['forecast'].replace([np.inf, -np.inf], np.nan)
    df['forecast'] = df['forecast'].fillna(df['bookings_on_hand'])
    return df

## Calculate error metrics (MAE, MAPE)
def compute_errors(df, naive_df=None):
    df = df.copy()
    df['error'] = df['final_demand'] - df['forecast']
    df['abs_error'] = df['error'].abs()
    df['mae'] = df['abs_error']
    df['mape'] = df['abs_error'] / df['final_demand'].replace(0, np.nan) * 100

        # --- Compute MASE ---
    if 'naive_forecast' in df.columns:
        # MAE of model
        mae_model = df['abs_error'].mean()

        # MAE of naive forecast
        df['naive_error'] = (df['final_demand'] - df['naive_forecast']).abs()
        mae_naive = df['naive_error'].mean()

        # Scaled error
        df['mase'] = df['abs_error'] / mae_naive if mae_naive != 0 else np.nan
    else:
        df['mase'] = np.nan
    return df


## Function for adding the forecast to the validation set, using days_prior 1-30
def validation(train, val, pickup, model_type='add'):
        # Merge pickup info by days_prior + hotel_code
    val = val[val['days_prior'].between(1, 30)].copy()
    booking_info = train[['days_prior', 'hotel_code', pickup]].drop_duplicates()
    forecast = val.merge(booking_info, on=['hotel_code', 'days_prior'], how='left')

    # Compute forecast based on model type
    if model_type == 'add':
        forecast['forecast'] = forecast['bookings_on_hand'] + forecast[pickup].fillna(0)
    elif model_type == 'multi':
        forecast['forecast'] = (forecast['bookings_on_hand'] / forecast[pickup].replace(0, np.nan))
    else:
        raise ValueError("Invalid model_type. Use 'add' or 'multi'.")

    # Compute errors relative to naive model (previous year's actuals)
    forecast = compute_errors(forecast, naive_df=train)

    return forecast

## Print out the forecasting errors grouped by hotel code, days_prior range, and month
def summarize_errors(df):
    # Overall metrics
    print(f"\nOVERALL METRICS:")
    print(f"MAE:  {df['mae'].mean():.2f}")
    print(f"MAPE: {df['mape'].mean():.2f}%")
    print(f"MASE: {df['mase'].mean():.2f}")

    # Group by month and hotel
    monthly_errors = (
        df.groupby(['hotel_code', 'month'])
        [['mae', 'mape', 'mase']]
        .mean()
        .reset_index()
    )

    # Group by days-prior range and hotel
    daysprior_errors = (
        df.groupby(['hotel_code', 'days_prior_range'])
        [['mae', 'mape', 'mase']]
        .mean()
        .reset_index()
    )

    # Display results
    for hotel in df['hotel_code'].unique():
        print(f"\n===== {hotel} =====")
        print("\nAverage Errors by Month:")
        print(
            monthly_errors[monthly_errors['hotel_code'] == hotel]
            .drop(columns='hotel_code')
            .rename(columns={'month': 'Month', 'mae': 'Average MAE', 'mape': 'Average MAPE', 'mase': 'Average MASE'})
            .to_string(index=False)
        )

        print("\nAverage Errors by Days-Prior Range:")
        print(
            daysprior_errors[daysprior_errors['hotel_code'] == hotel]
            .drop(columns='hotel_code')
            .rename(columns={'days_prior_range': 'Days-Prior Range', 'mae': 'Average MAE', 'mape': 'Average MAPE', 'mase': 'Average MASE'})
            .to_string(index=False)
        )

##### Apply the functions above to the dataframe in training and validation data

In [ ]:
## Train the data and compute errors in train
train_add = add_model(train)
train_add = compute_errors(train_add)

train_multi = multi_model(train)
train_multi = compute_errors(train_multi)

## Validate models with test/validation data
val_add = validation(train_add, val, 'average_remaining_demand', model_type='add')
val_multi = validation(train_multi, val, 'average_booking_rate', model_type='multi')


NameError: name 'add_model' is not defined

#####

##### Results from ADDITIVE model with forcasting errors below

In [ ]:
summarize_errors(val_add)

##### Results from MULTIPLICATIVE model with forcasting errors below

In [ ]:
summarize_errors(val_multi)

##### **2. Define Neural Network Model**

In [ ]:
###########################################################################
### Define Neural Network Model
###########################################################################

class Model(nn.Module):  ## subclass nn.Module
    ## input size: number of features (X variables), dimension of X
    ## hidden-size: number of nodes in hidden layers
    ## output_size: dimension of y
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        ## nn.Linear: a linear transformation,a.k.a a fully connected layer or a dense layer to implement summation junction
        ## define two hidden layers, layer1 and layer2
        self.layer1 = nn.Linear(input_size, hidden_size)   ## layer 1 has "input_size" number of input nodes and "hidden_size" number of output nodes
        self.layer2 = nn.Linear(hidden_size, output_size)  ## layer 2 has "hidden_size" number of input nodes and "output_size" number of output nodes
        self.activation = nn.ReLU()  # Use ReLU for hidden layer

    ## forward() method that must be defined within any class that subclasses torch.nn.Module.
    ## This method defines the computational graph of a neural network,
    ## outlining how input data is processed through the network's layers to produce an output.
    def forward(self, x):
        x = self.activation(self.layer1(x))  # Apply ReLU to hidden layer
        x = self.layer2(x)                   # Linear output for regression
        return x

In [ ]:
###########################################################################
### Helper functions, validation with days prior 1-30
###########################################################################

def prepare_data(train_df, val_df, feature_cols, target_col):
    """Prepare train and val sets"""
    val_df = val_df[val_df['days_prior'].between(1, 30)].copy()
    X_train = pd.get_dummies(train_df[feature_cols], columns=["day_of_week"], drop_first=True)
    X_val = pd.get_dummies(val_df[feature_cols], columns=["day_of_week"], drop_first=True)
    X_train, X_val = X_train.align(X_val, join="left", axis=1, fill_value=0)

    y_train = train_df[target_col].to_numpy().astype(np.float32)
    y_val = val_df[target_col].to_numpy().astype(np.float32)

    return X_train.to_numpy().astype(np.float32), y_train, X_val.to_numpy().astype(np.float32), y_val

##### Normalize data, convert to tensors, create DataLoader, setup loss function and optimizer, train model, and evaluate

In [ ]:
def train_model(X_train, y_train, X_test, y_test, num_epochs=10, hidden_size=20, lr=0.001):
    """Train and evaluate model for one hotel."""
    # Normalize
    X_train_norm = (X_train - np.mean(X_train)) / np.std(X_train)
    X_test_norm = (X_test - np.mean(X_train)) / np.std(X_train)

    # Convert to tensors
    X_train_t = torch.from_numpy(X_train_norm).float()
    y_train_t = torch.from_numpy(y_train).float()
    X_test_t = torch.from_numpy(X_test_norm).float()
    y_test_t = torch.from_numpy(y_test).float()

    # Create DataLoader
    train_ds = TensorDataset(X_train_t, y_train_t)
    train_dl = DataLoader(train_ds, batch_size=20, shuffle=True)

    # Model setup
    input_size = X_train_t.shape[1]
    model = Model(input_size, hidden_size, 1)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Train
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred.squeeze(), yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_dl):.4f}")

    # Evaluate
    model.eval()
    with torch.no_grad():
        preds = model(X_test_t).squeeze()
        mse = loss_fn(preds, y_test_t)
        rmse = torch.sqrt(mse).item()

    print(f"Validation RMSE: {rmse:.4f}")
    return model, preds.numpy(), rmse


##### Loop through hotels and run model + error summaries

In [ ]:
###########################################################################
### Loop through hotels and run model + error summaries
###########################################################################
all_results = []

for hotel in df['hotel_code'].unique():
    print(f"\n== {hotel} ==")
    train_hotel = train[train['hotel_code'] == hotel]
    val_hotel = val[val['hotel_code'] == hotel]

    if len(train_hotel) < 10 or len(val_hotel) < 10:
        print("Not enough data, skipping...")
        continue

    # Prepare data
    X_train, y_train, X_val, y_val = prepare_data(train_hotel, val_hotel, feature_cols, target_col)

    # Train model
    model, y_pred, rmse = train_model(X_train, y_train, X_val, y_val)

    # Compute and store errors
    val_hotel = val_hotel[val_hotel['days_prior'].between(1, 30)].copy()
    val_hotel = val_hotel.copy()
    val_hotel['pred'] = y_pred
    val_hotel['abs_error'] = np.abs(val_hotel['pred'] - val_hotel['final_demand'])
    val_hotel['mae'] = val_hotel['abs_error']
    val_hotel['mape'] = val_hotel['abs_error'] / (val_hotel['final_demand'] + 1e-8)
    val_hotel['rmse'] = rmse

    # Compute MASE
    val_hotel = compute_mase(val_hotel, train_hotel)

    # Append
    all_results.append(val_hotel)

# Combine all hotel results
final_results = pd.concat(all_results, ignore_index=True)
print("\All models: ")

# Summarize errors across all hotels
summarize_errors(final_results)

<>:38: SyntaxWarning: invalid escape sequence '\A'
<>:38: SyntaxWarning: invalid escape sequence '\A'
/tmp/ipython-input-750035624.py:38: SyntaxWarning: invalid escape sequence '\A'
  print("\All models: ")



== G ==
Epoch 1/10, Loss: 1229.9921
Epoch 2/10, Loss: 647.4254
Epoch 3/10, Loss: 631.5223
Epoch 4/10, Loss: 622.6189
Epoch 5/10, Loss: 617.6654
Epoch 6/10, Loss: 613.7890
Epoch 7/10, Loss: 610.0596
Epoch 8/10, Loss: 606.2392
Epoch 9/10, Loss: 601.8015
Epoch 10/10, Loss: 597.8620
Validation RMSE: 17.6850

== M ==
Epoch 1/10, Loss: 2718.0788
Epoch 2/10, Loss: 2302.4971
Epoch 3/10, Loss: 2288.5612
Epoch 4/10, Loss: 2276.8740
Epoch 5/10, Loss: 2266.5360
Epoch 6/10, Loss: 2256.8505
Epoch 7/10, Loss: 2247.1827
Epoch 8/10, Loss: 2237.3794
Epoch 9/10, Loss: 2227.0584
Epoch 10/10, Loss: 2215.6422
Validation RMSE: 40.4877

== W ==
Epoch 1/10, Loss: 1592.6196
Epoch 2/10, Loss: 1285.5900
Epoch 3/10, Loss: 1269.8231
Epoch 4/10, Loss: 1256.4038
Epoch 5/10, Loss: 1242.0344
Epoch 6/10, Loss: 1226.7056
Epoch 7/10, Loss: 1211.0253
Epoch 8/10, Loss: 1196.2281
Epoch 9/10, Loss: 1184.1455
Epoch 10/10, Loss: 1173.6146
Validation RMSE: 24.3681
\All models: 

OVERALL METRICS:
MAE:  23.10
MAPE: 0.35%
MASE: 1.

/tmp/ipython-input-195076159.py:105: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['hotel_code', 'days_prior_range'])



##### **Time Series LSTM Model**

In [ ]:
###########################################################################
### Define neural network model class by subclassing nn.Module
###########################################################################
##### Define a LSTM Model - LSTM layer + linear layer

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1, :])
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') ## get the device information
print(device)

##### Reshape data and convert to tensor

In [ ]:
def create_sequences(data, lookback=100):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback])
    return np.array(X), np.array(y)

def prepare_data(df, hotel_code):
    hotel_df = df[df["hotel_code"] == hotel_code].copy()
    hotel_df = hotel_df.set_index("stay_date").asfreq("D").fillna(method="ffill")
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(hotel_df[["final_demand"]])
    return hotel_df, scaled, scaler

##### Loop through each hotel and apply Time Series LSTM

In [ ]:
# Filter relevant columns
df = df[["stay_date", "final_demand", "hotel_code"]].drop_duplicates()

# Split train/test
train_df = df[df["stay_date"] < "2010-01-01"]
test_df  = df[(df["stay_date"] >= "2010-01-01") & (df["stay_date"] <= "2010-04-30")]

# Combine for continuous indexing (needed for lookback)
df = pd.concat([train_df, test_df]).sort_values("stay_date")


In [ ]:
input_size = 1
num_layers = 2
hidden_size = 100
output_size = 1
dropout = 0  # Added dropout for regularization

model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)

In [ ]:
# # @title
# lookback = 50
# forecast_horizon = 30
# asof_start = pd.to_datetime("2010-01-30")
# asof_end   = pd.to_datetime("2010-03-31")

# asof_dates = pd.date_range(asof_start, asof_end, freq="D")

# results = []

# for hotel in df["hotel_code"].unique():
#     hotel_df, scaled, scaler = prepare_data(df, hotel)
#     demand_series = scaled.flatten()

#     for asof_date in asof_dates:
#         # Only use data up to as-of date
#         mask = hotel_df.index <= asof_date
#         train_series = demand_series[mask]

#         if len(train_series) < lookback:
#             continue  # skip if not enough history

#         # Create sequences
#         X_train, y_train = create_sequences(train_series, lookback)
#         X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)
#         y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)

#         # Use the model initialized outside the loop
#         # model = LSTMModel() # Removed this line
#         criterion = nn.MSELoss()
#         optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

#         model.train()
#         for epoch in range(20):
#             optimizer.zero_grad()
#             y_pred = model(X_train)
#             loss = criterion(y_pred, y_train)
#             loss.backward()
#             optimizer.step()

#         # Forecast next 30 days iteratively
#         model.eval()
#         last_input = train_series[-lookback:].reshape(1, lookback, 1)
#         last_input = torch.tensor(last_input, dtype=torch.float32)
#         preds = []

#         for dp in range(forecast_horizon):
#             with torch.no_grad():
#                 pred = model(last_input).item()
#                 preds.append(pred)
#                 last_input = torch.cat([last_input[:, 1:, :],
#                                         torch.tensor([[[pred]]])], dim=1)

#         preds_rescaled = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()
#         forecast_dates = pd.date_range(asof_date + pd.Timedelta(days=1),
#                                        asof_date + pd.Timedelta(days=forecast_horizon))

#         results.append(pd.DataFrame({
#             "hotel_code": hotel,
#             "asof_date": asof_date,
#             "forecast_date": forecast_dates,
#             "forecast_demand": preds_rescaled
#         }))

/tmp/ipython-input-2720528048.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hotel_df = hotel_df.set_index("stay_date").asfreq("D").fillna(method="ffill")


KeyboardInterrupt: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

lookback = 50
forecast_horizon = 30
asof_dates = pd.date_range("2010-01-30", "2010-03-31", freq="D")

results = []

for hotel in df["hotel_code"].unique():
    print(f"\nTraining once for hotel {hotel} ...")
    hotel_df, scaled, scaler = prepare_data(df, hotel)
    demand_series = scaled.flatten()

    # Train only up to 2010-01-01
    train_mask = hotel_df.index < "2010-01-01"
    train_series = demand_series[train_mask]
    X_train, y_train = create_sequences(train_series, lookback)

    X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1).to(device)
    y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1).to(device)

    input_size = 1
    num_layers = 5
    hidden_size = 32
    output_size = 1
    dropout = 0.2

    model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size, dropout=dropout).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    print("⏳ Training model ...")
    model.train()
    for epoch in range(20):
        optimizer.zero_grad()
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
    print("✅ Done training.\n")

    # --- Rolling as-of-date forecasts ---
    model.eval()
    for asof_date in tqdm(asof_dates, desc=f"Rolling forecasts for {hotel}"):
        # use all data available up to that as-of date
        mask = hotel_df.index <= asof_date
        full_series = demand_series[mask]

        if len(full_series) < lookback:
            continue

        last_input = full_series[-lookback:].reshape(1, lookback, 1)
        last_input = torch.tensor(last_input, dtype=torch.float32).to(device)
        preds = []

        for dp in range(forecast_horizon):
            with torch.no_grad():
                pred = model(last_input).item()
                preds.append(pred)
                new_input = torch.tensor([[[pred]]], dtype=torch.float32).to(device)
                last_input = torch.cat([last_input[:, 1:, :], new_input], dim=1)

        preds_rescaled = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()
        forecast_dates = pd.date_range(asof_date + pd.Timedelta(days=1),
                                       asof_date + pd.Timedelta(days=forecast_horizon))

        tmp = pd.DataFrame({
            "hotel_code": hotel,
            "as_of_date": asof_date,
            "stay_date": forecast_dates,
            "days_prior": np.arange(1, forecast_horizon + 1),
            "fcst": preds_rescaled
        })
        results.append(tmp)

forecast_df = pd.concat(results).reset_index(drop=True)

Using cuda

Training once for hotel G ...


/tmp/ipython-input-2720528048.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hotel_df = hotel_df.set_index("stay_date").asfreq("D").fillna(method="ffill")


In [ ]:
forecast_df = pd.concat(results).reset_index(drop=True)

# Merge with actual demand
actuals = df[["stay_date", "hotel_code", "final_demand"]]
forecast_df = forecast_df.merge(actuals, left_on=["forecast_date","hotel_code"],
                                right_on=["stay_date","hotel_code"], how="left")

# Compute errors
forecast_df["abs_error"] = abs(forecast_df["forecast_demand"] - forecast_df["final_demand"])
forecast_df["mape"] = forecast_df["abs_error"] / (forecast_df["final_demand"] + 1e-8)


In [ ]:
# @title
# ###########################################################################
# ### Main loop over hotel codes
# ###########################################################################
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print("Using device:", device)

# # Prepare data
# df = df[["stay_date", "final_demand", "hotel_code"]].drop_duplicates()
# df["stay_date"] = pd.to_datetime(df["stay_date"])
# hotel_codes = df["hotel_code"].unique()

# # Loop through each hotel
# for code in hotel_codes:
#     print(f"\n========== HOTEL: {code} ==========")
#     hotel_df = df[df["hotel_code"] == code].sort_values("stay_date")

#     # Split train/test by date
#     train_data = hotel_df[hotel_df["stay_date"] < "2010-01-01"].copy()
#     test_data = hotel_df[hotel_df["stay_date"] >= "2010-01-01"].copy()

#     # Prepare dataset (only final_demand)
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     dataset_train = scaler.fit_transform(train_data[["final_demand"]])
#     dataset_test = scaler.transform(test_data[["final_demand"]])

#     # Prepare dataloaders
#     sequence_length = 30
#     batch_size = 32

#     train_loader = prepare_timeseries_dataloader(dataset_train, sequence_length, batch_size, shuffle=True)
#     test_loader = prepare_timeseries_dataloader(dataset_test, sequence_length, batch_size, shuffle=False)

#     # Define model
#     input_size = 1
#     hidden_size = 128
#     num_layers = 2
#     output_size = 1
#     dropout = 0.2

#     model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout).to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     loss_fn = nn.MSELoss()

#     # Train model
#     num_epochs = 50
#     for epoch in range(num_epochs):
#         model.train()
#         total_loss = 0
#         for batch_X, batch_y in train_loader:
#             batch_X, batch_y = batch_X.to(device), batch_y.to(device)
#             optimizer.zero_grad()
#             preds = model(batch_X)
#             loss = loss_fn(preds, batch_y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()

#         if (epoch + 1) % 10 == 0:
#             avg_loss = total_loss / len(train_loader)
#             print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_loss:.4f}")

#     ###########################################################################
#     ### Forecast
#     ###########################################################################
#     model.eval()
#     test_series = dataset_test.flatten()
#     last_sequence = test_series[-sequence_length:]  # last 30 days
#     forecasted_values = []

#     with torch.no_grad():
#         for _ in range(30):  # predict next 30 days
#             seq_input = torch.tensor(last_sequence).float().view(1, -1, 1).to(device)
#             pred = model(seq_input).cpu().item()
#             forecasted_values.append(pred)
#             last_sequence = np.append(last_sequence[1:], pred)

#     # Inverse transform
#     forecasted_values = scaler.inverse_transform(np.array(forecasted_values).reshape(-1, 1)).flatten()
#     actual_values = test_data["final_demand"].values[-30:]
#     future_dates = pd.date_range(start=test_data["stay_date"].iloc[-1] + pd.Timedelta(days=1), periods=30)

Using device: cuda

========== HOTEL: G ==========
Epoch [10/50] - Train Loss: 0.0258


KeyboardInterrupt: 

##### **LTSM**